# <center> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных соревнования. Знакомиться будем с помощью [данных](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) Scikit-learn по новостям, сначала в режиме бинарной классификации, затем – в многоклассовом режиме. Далее будем классифицировать рецензии к фильмам с сайта IMDB. Наконец, применим Vowpal Wabbit к нашему соревнованию. Материала немало, но Vowpal Wabbit того стоит!

**План 6 недели:**
- Часть 1. Тьюториал по Vowpal Wabbit. Новости. Бинарная классификация
- Часть 2. Тьюториал по Vowpal Wabbit. Новости. Многоклассовая классификация
- Часть 3. Тьюториал по Vowpal Wabbit. Рецензии к фильмам IMDB
- Часть 4. Применение Vowpal Wabbit к данным по посещению сайтов


**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

## Часть 1. Тьюториал по Vowpal Wabbit. Новости. Бинарная классификация

Vowpal Wabbit (VW) является одной из наиболее широко используемых библиотек в индустрии. Её отличает высокая скорость работы и поддержка большого количества различных режимов обучения. Особый интерес для больших и высокоразмерных данных представляет онлайн-обучение – самая сильная сторона библиотеки. 


Основным интерфейсом для работы с VW является shell.

In [5]:
import os
import re
import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%pylab inline
import sklearn.datasets
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

Populating the interactive namespace from numpy and matplotlib


Для того, чтобы изучить возможные режимы работы vw, воспользуемся справкой:

In [ ]:
!vw --help

Vowpal Wabbit считывает данные из файла или стандартного ввода (stdin) в формате, который имеет следующий вид:

`[Label] [Importance] [Tag]|Namespace Features |Namespace Features ... |Namespace Features`

`Namespace=String[:Value]`

`Features=(String[:Value] )*`

где [] обозначает необязательные элементы, а (...)\* означает повтор неопределенное число раз. 

- **Label** является числом, "правильным" ответом. В случае классификации обычно принимает значение 1/-1, а в случае регрессии некоторое вещественное число
- **Importance** является числом и отвечает за вес примера при обучении. Это позволяет бороться с проблемой несбалансированных данных, изученной нами ранее
- **Tag** является некоторой строкой без пробелов и отвечает за некоторое "название" примера, которое сохраняется при предсказании ответа. Для того, чтобы отделить Tag от Importance лучше начинать Tag с символа '.
- **Namespace** служит для создания отдельных пространств признаков. В аргументах Namespace именуются по первой букве, это нужно учитывать при выборе их названий
- **Features** являются непосредственно признаками объекта внутри **Namespace**. Признаки по умолчанию имеют вес 1.0, но его можно переопределить, к примеру feature:0.1. 


К примеру, под такой формат подходит следующая строка:

```
1 1.0 |Subject WHAT car is this |Organization University of Maryland:0.5 College Park
```


чтобы убедиться в этом, запустим vw с этим обучающим примером:

In [ ]:
! echo '1 1.0 |Subject WHAT car is this |Organization Maryland:0.5 College Park' | vw

VW является прекрасным инструментом для работы с текстовыми данными. Убедимся в этом с помощью выборки 20newsgroups, содержащей письма из 20 различных тематических рассылок:

In [ ]:
newsgroups = sklearn.datasets.fetch_20newsgroups('news_data/')

In [ ]:
newsgroups['target_names']

Рассмотрим первый текстовый документ этой коллекции:

In [ ]:
text = newsgroups['data'][0]
target = newsgroups['target_names'][newsgroups['target'][0]]

print('-----')
print(target)
print('-----')
print(text.strip())
print('----')

**Приведем данные к формату Vowpal Wabbit, при этом оставляя только слова не короче 3 символов. Здесь мы не выполняем многие важные в анализе текстов процедуры (стемминг и лемматизацию), но, как увидим, задача и так будет решаться хорошо.**

In [ ]:
def to_vw_format(document, label=None):
    return str(label or '') + ' |text ' + ' '.join(re.findall('\w{3,}', 
                                                              document.lower())) + '\n'

to_vw_format(text, 1 if target == 'rec.autos' else -1)

Разобьем выборку на обучающую и тестовую и запишем в файл преобразованные таким образом документы. Будем считать документ положительным, если он относится к рассылке про автомобили **rec.autos**. Так мы построим модель, отличающую письма про автомобили от остальных: 

In [ ]:
all_documents = newsgroups['data']
all_targets = [1 if newsgroups['target_names'][target] == 'rec.autos' 
               else -1 for target in newsgroups['target']]

In [ ]:
train_documents, test_documents, train_labels, test_labels = \
    train_test_split(all_documents, all_targets, random_state=7)
    
with open('news_data/20news_train.vw', 'w') as vw_train_data:
    for text, target in zip(train_documents, train_labels):
        vw_train_data.write(to_vw_format(text, target))
with open('news_data/20news_test.vw', 'w') as vw_test_data:
    for text in test_documents:
        vw_test_data.write(to_vw_format(text))

Запустим Vowpal Wabbit на сформированном файле. Мы решаем задачу классификации, поэтому зададим функцию потерь в значение hinge (линейный SVM). Построенную модель мы сохраним в соответствующий файл 20news_model.vw:

In [ ]:
!vw -d news_data/20news_train.vw --loss_function hinge -f news_data/20news_model.vw

Модель обучена. VW выводит достаточно много полезной информации по ходу обучения. Обратите внимание, что average loss снижался по ходу выполнения итераций. Для вычисления функции потерь VW использует еще не просмотренные примеры, поэтому, как правило, эта оценка является корректной. Применим обученную модель на тестовой выборке, сохраняя предсказания в файл с помощью опции -p: 

In [ ]:
!vw -i news_data/20news_model.vw -t -d news_data/20news_test.vw \
-p news_data/20news_test_predictions.txt

Загрузим полученные предсказания, вычислим AUC и отобразим ROC-кривую:

In [ ]:
with open('news_data/20news_test_predictions.txt') as pred_file:
    test_prediction = [float(label) 
                             for label in pred_file.readlines()]

auc = sklearn.metrics.roc_auc_score(test_labels, test_prediction)
roc_curve = sklearn.metrics.roc_curve(test_labels, test_prediction)

with plt.xkcd():
    plt.plot(roc_curve[0], roc_curve[1]);
    plt.plot([0,1], [0,1])
    plt.xlabel('FPR'); plt.ylabel('TPR'); 
    plt.title('test AUC = %f' % (auc)); 
    plt.axis([-0.05,1.05,-0.05,1.05]);

Полученное значения AUC говорит о высоком качестве классификации.

## Часть 2. Тьюториал по Vowpal Wabbit. Новости. Многоклассовая классификация

**Используем ту же выборку, что в прошлой части, но решаем задачу многоклассовой классификации. Тут Vowpal Wabbit слегка капризничает – он любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 20). Поэтому придется применить LabelEncoder, да еще и +1 потом добавить (LabelEncoder переводит метки в диапазон от 0 до K-1).**

In [ ]:
all_documents = newsgroups['data']
topic_encoder = LabelEncoder()
all_targets_mult = topic_encoder.fit_transform(newsgroups['target']) + 1

**Выборки будут те же, а метки поменяются, train_labels_mult и test_labels_mult – векторы меток от 1 до 20.**

In [ ]:
train_documents, test_documents, train_labels_mult, test_labels_mult = \
    train_test_split(all_documents, all_targets_mult, random_state=7)

In [ ]:
with open('news_data/20news_train_mult.vw', 'w') as vw_train_data:
    for text, target in zip(train_documents, train_labels_mult):
        vw_train_data.write(to_vw_format(text, target))
with open('news_data/20news_test_mult.vw', 'w') as vw_test_data:
    for text in test_documents:
        vw_test_data.write(to_vw_format(text))

**Обучим Vowpal Wabbit в режиме многоклассовой классификации, передав параметр *oaa* (от "one against all"), равный числу классов. Также перечислим параметры, которые можно понастраивать, и от которых качество модели может довольно значительно зависеть (более полно – в официальном [тьюториале](https://github.com/JohnLangford/vowpal_wabbit/wiki/Tutorial) по Vowpal Wabbit):**
 - темп обучения (-l, по умолчанию 0.5) – коэффициент перед изменением весов модели при каждом изменении
 - степень убывания темпа обучения (--power_t, по умолчанию 0.5) – на практике проверено, что если темп обучения уменьшается при увеличении числа итераций стохастического градиентного спуска, то минимум функции находится лучше 
 - функция потерь (--loss_function) – от нее, по сути, зависит обучаемый алгоритм
 - регуляризация (-l1) – тут надо обратить внимание на то, что в VW регуляризация считается для каждого объекта, поэтому коэффициенты регуляризации обычно берутся малыми, около $10^{-20}.$
 
 **Дополнительно: в соревновании можно попробовать автоматическую настройку параметров Vowpal Wabbit с Hyperopt. Пока это работает только с Python 2. [Статья](https://habrahabr.ru/company/dca/blog/272697/) на Хабре.**

In [ ]:
%%time
!vw --oaa 20 news_data/20news_train_mult.vw -f news_data/20news_model_mult.vw \
--loss_function=hinge --quiet

In [ ]:
%%time
!vw -i news_data/20news_model_mult.vw -t -d news_data/20news_test_mult.vw \
-p news_data/20news_test_predictions_mult.txt --quiet

In [ ]:
with open('news_data/20news_test_predictions_mult.txt') as pred_file:
    test_prediction_mult = [float(label) 
                             for label in pred_file.readlines()]

In [ ]:
accuracy_score(test_labels_mult, test_prediction_mult)

**Выведем раскрашенную матрицу ошибок полученного классификатора.**

In [ ]:
from sklearn.metrics import confusion_matrix

M = confusion_matrix(test_labels_mult, test_prediction_mult)
M_normalized = M.astype('float') / M.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(10,10))
im = plt.imshow(M_normalized, interpolation='nearest')
plt.colorbar(im, shrink=0.71)
tick_marks = np.arange(len(newsgroups['target_names']))
plt.xticks(tick_marks - 0.5, newsgroups['target_names'], rotation=45)
plt.yticks(tick_marks, newsgroups['target_names'])
plt.tight_layout()
plt.ylabel('True topic')
plt.xlabel('Predicted topic')
plt.title('Normalized confusion matrix')
plt.show()

## Часть 3. Тьюториал по Vowpal Wabbit. Рецензии к фильмам IMDB

**В этой части мы будем заниматься бинарной классификацией отзывов к фильмам, опубликованным на сайте IMDB. Обратите внимание, насколько быстро будет работать Vowpal Wabbit.**

**Используем функцию *load_files* из sklearn.datasets для загрузки отзывов по фильмам [отсюда](https://yadi.sk/d/Tg1Tflur333iLr). Скачайте данные и положите рядом с этой тетрадкой в каталог *imdb_reviews* (в нем должны быть каталоги *train* и *test*). Разархивирование может занять несколько минут – там 100 тыс. файлов. В обучающей и тестовой выборках по 12500 тысяч хороших и плохих отзывов к фильмам. Отделим данные (собственно тексты) от меток.**

In [ ]:
from sklearn.datasets import load_files

In [ ]:
reviews_train = load_files('imdb_reviews/train/')
text_train, y_train = reviews_train.data, reviews_train.target

In [ ]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))

То же самое с тестовой выборкой.

In [ ]:
reviews_test = load_files('imdb_reviews/test/')
text_test, y_test = reviews_test.data, reviews_train.target
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

**Примеры отзывов и соответствующих меток.**

In [ ]:
text_train[0]

In [ ]:
y_train[0] # хороший отзыв

In [ ]:
text_train[1]

In [ ]:
y_train[1] # плохой отзыв

**Будем использовать ранее написанную функцию to_vw_format.**

In [ ]:
to_vw_format(str(text_train[1]), 1 if y_train[0] == 1 else -1)

**Подготовим обучающую (movie_reviews_train.vw), отложенную (movie_reviews_valid.vw) и тестовую (movie_reviews_test.vw) выборки для Vowpal Wabbit. 70% исходной обучаюшей выборки оставим под обучение, 30% – под отложенную выборку.**

In [ ]:
train_share = int(0.7 * len(text_train))
train, valid = text_train[:train_share], text_train[train_share:]
train_labels, valid_labels = y_train[:train_share], y_train[train_share:]

In [ ]:
len(train_labels), len(valid_labels)

In [ ]:
with open('imdb_reviews/movie_reviews_train.vw', 'w') as vw_train_data:
    for text, target in zip(train, train_labels):
        vw_train_data.write(to_vw_format(str(text), 1 if target == 1 else -1))
with open('imdb_reviews/movie_reviews_valid.vw', 'w') as vw_train_data:
    for text, target in zip(valid, valid_labels):
        vw_train_data.write(to_vw_format(str(text), 1 if target == 1 else -1))
with open('imdb_reviews/movie_reviews_test.vw', 'w') as vw_test_data:
    for text in text_test:
        vw_test_data.write(to_vw_format(str(text)))

In [ ]:
!head -2 imdb_reviews/movie_reviews_train.vw

In [ ]:
!head -2 imdb_reviews/movie_reviews_valid.vw

In [ ]:
!head -2 imdb_reviews/movie_reviews_test.vw

**Обучим модель Vowpal Wabbit со следующими аргументами:**

 - -d, путь к обучающей выборке (соотв. файл .vw )
 - --loss_function – hinge (хотя можно и поэкспериментировать с другими)
 - -f – путь к файлу, в который запишется модель (можно тоже в формате .vw)

In [ ]:
!vw -d imdb_reviews/movie_reviews_train.vw \
--loss_function hinge -f imdb_reviews/movie_reviews_model.vw --quiet

**Сделаем прогноз для отложенной выборки с помощью обученной модели Vowpal Wabbit, передав следующие аргументы:**
 - -i –путь к обученной модели (соотв. файл .vw)
 - -t -d – путь к отложенной выборке (соотв. файл .vw)
 - -p – путь к txt-файлу, куда запишутся прогнозы

In [ ]:
!vw -i imdb_reviews/movie_reviews_model.vw -t -d imdb_reviews/movie_reviews_valid.vw \
-p imdb_reviews/movie_valid_pred.txt --quiet

**Считаем прогноз из файла и посчитаем долю правильных ответов и ROC AUC. Учтем, что VW выводит оценки вероятности принадлежности к классу +1. Эти оценки распределены на [-1, 1], поэтому бинарным ответом алгоритма (0 или 1) будем попросту считать тот факт, что оценка получилась положительной.**

In [ ]:
with open('imdb_reviews/movie_valid_pred.txt') as pred_file:
    valid_prediction = [float(label) 
                             for label in pred_file.readlines()]
print("Accuracy: {}".format(round(accuracy_score(valid_labels, 
               [int(pred_prob > 0) for pred_prob in valid_prediction]), 3)))
print("AUC: {}".format(round(roc_auc_score(valid_labels, valid_prediction), 3)))

**Сделаем то же самое для тестовой выборки.**

In [ ]:
!vw -i imdb_reviews/movie_reviews_model.vw -t -d imdb_reviews/movie_reviews_test.vw \
-p imdb_reviews/movie_test_pred.txt --quiet

In [ ]:
with open('imdb_reviews/movie_test_pred.txt') as pred_file:
    test_prediction = [float(label) 
                             for label in pred_file.readlines()]
print("Accuracy: {}".format(round(accuracy_score(y_test, 
               [int(pred_prob > 0) for pred_prob in test_prediction]), 3)))
print("AUC: {}".format(round(roc_auc_score(y_test, test_prediction), 3)))

**Попробуем улучшить прогноз за счет задействования биграмм.**

In [ ]:
!vw -d imdb_reviews/movie_reviews_train.vw \
--loss_function hinge --ngram 2 -f imdb_reviews/movie_reviews_model2.vw --quiet

In [ ]:
!vw -i imdb_reviews/movie_reviews_model2.vw -t -d imdb_reviews/movie_reviews_valid.vw \
-p imdb_reviews/movie_valid_pred2.txt --quiet

In [ ]:
with open('imdb_reviews/movie_valid_pred2.txt') as pred_file:
    valid_prediction = [float(label) 
                             for label in pred_file.readlines()]
print("Accuracy: {}".format(round(accuracy_score(valid_labels, 
               [int(pred_prob > 0) for pred_prob in valid_prediction]), 3)))
print("AUC: {}".format(round(roc_auc_score(valid_labels, valid_prediction), 3)))

In [ ]:
!vw -i imdb_reviews/movie_reviews_model2.vw -t -d imdb_reviews/movie_reviews_test.vw \
-p imdb_reviews/movie_test_pred2.txt --quiet

In [ ]:
with open('imdb_reviews/movie_test_pred2.txt') as pred_file:
    test_prediction2 = [float(label) 
                             for label in pred_file.readlines()]
print("Accuracy: {}".format(round(accuracy_score(y_test, 
               [int(pred_prob > 0) for pred_prob in test_prediction2]), 3)))
print("AUC: {}".format(round(roc_auc_score(y_test, test_prediction2), 3)))

Видим, что биграммы помогли повысить качество классификации.

## Часть 4. Применение Vowpal Wabbit к данным по посещению сайтов

**Загрузим созданные ранее pickle-объекты, соответствующие разреженным данным [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data), которые мы создали на прошлой неделе.**

In [6]:
import pickle

In [7]:
with open('kaggle_data/X_train_sparse.pkl', 'rb') as X_train_sparse_pkl:
    X_train_sparse = pickle.load(X_train_sparse_pkl)
with open('kaggle_data/X_test_sparse.pkl', 'rb') as X_test_sparse_pkl:
    X_test_sparse = pickle.load(X_test_sparse_pkl)
with open('kaggle_data/train_target.pkl', 'rb') as train_target_pkl:
    y = pickle.load(train_target_pkl)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 550). Поэтому придется применить LabelEncoder, да еще и +1 потом добавить (LabelEncoder переводит метки в диапозон от 0 до K-1).**

In [8]:
class_encoder = LabelEncoder().fit(y.astype('str'))
y_for_vw = class_encoder.transform(y.astype('str')) + 1

**Выделим обучающую и оставленную части исходной обучающей выборки.**

In [9]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y_for_vw, test_size=0.3, 
                                                     random_state=7, stratify=y_for_vw)

**Реализуйте функцию, переводящую разреженную матрицу в формат Vowpal Wabbit.**

Вход:
 - X_sparse – разреженная матрица SciPy.sparse.csr_matrix
 - y (необяз.) – вектор ответов. Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- можно делать по-разному, но скорее всего помогут атрибуты *data* и *indices* разреженной матрицы. Обратите внимание на особенность (или баг?) csr_matrix: если признак 8 попадается 3 раза, ему в паре *data* и *indices* могут быть соответвствовать как [3] в *data* и [8] в *indices*, так и [1,1,1] в *data* и [8,8,8] в *indices*.
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [ ]:
def sparse_row_to_vw(x_row, y_row):
    x_label = " ".join(
                ["{}:{}".format(int(site) + 1, int(count)) for site, count in zip(x_row.indices, x_row.data)])
    res = str(y_row or "") + " | sites " + x_label + "\n"
    return res

In [ ]:
def sparse_matrix_to_vw(X_sparse, y=None, out_file='tmp.vw'):
    if y is None:
        y = [1] * X_sparse.shape[0]
    with open(out_file, "w") as f_out:
        for x_row, y_row in zip(X_sparse, y):
            f_out.write(sparse_row_to_vw(x_row, y_row))

**Примените написанную функцию к части обучающей выборки (X_train, y_train), к отложенной выборке (X_valid, y_valid), ко всей обучающей выборке (X_train_sparse, y_for_vw) и ко всей тестовой выборке X_test_sparse.**

In [ ]:
%%time
sparse_matrix_to_vw(X_train, y_train, 'kaggle_data/train_part.vw')
sparse_matrix_to_vw(X_valid, y_valid, 'kaggle_data/valid.vw')
sparse_matrix_to_vw(X_train_sparse, y_for_vw, 'kaggle_data/train.vw')
sparse_matrix_to_vw(X_test_sparse, out_file='kaggle_data/test.vw')

**Первые 3 строки из каждого файла должны получиться примерно такими (с точностью до нумерации сайтов). **

In [ ]:
!head -3 kaggle_data/train_part.vw

In [ ]:
!head -3 kaggle_data/valid.vw

In [ ]:
!head -3 kaggle_data/train.vw

In [ ]:
!head -3 kaggle_data/test.vw

**Обучим модель Vowpal Wabbit со следующими аргументами:**

 - -d, путь к обучающей выборке (соотв. файл .vw )
 - --loss_function – hinge (хотя можно и поэкспериментировать с другими)
 - -f – путь к файлу, в который запишется модель (можно тоже в формате .vw)

In [ ]:
!vw -d imdb_reviews/movie_reviews_train.vw \
--loss_function hinge -f imdb_reviews/movie_reviews_model.vw --quiet

**Сделаем прогноз для отложенной выборки с помощью обученной модели Vowpal Wabbit, передав следующие аргументы:**
 - -i –путь к обученной модели (соотв. файл .vw)
 - -t -d – путь к отложенной выборке (соотв. файл .vw)
 - -p – путь к txt-файлу, куда запишутся прогнозы

In [ ]:
!vw -i imdb_reviews/movie_reviews_model.vw -t -d imdb_reviews/movie_reviews_valid.vw \
-p imdb_reviews/movie_valid_pred.txt --quiet

**Обучите модель на выборке kaggle_data/train_part.vw. Укажите, что решается задача классификации с 550 классами (*--oaa*), сделайте 10 проходов по выборке (*--passes*). Задайте некоторый кэш-файл (*--cache_file*), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента *-k*). Также укажите значение параметра b=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [3]:
%%time
!vw --oaa 550 -d kaggle_data/train_part.vw --passes 10 -b 26 --cache_file kaggle_data/train_part.vw_cache -k \
-f kaggle_data/users_model.vw --quiet 

CPU times: user 407 ms, sys: 20 ms, total: 427 ms
Wall time: 22.8 s


**Запишите прогнозы на выборке *kaggle_data/valid.vw* в *kaggle_data/vw_valid_pred.csv*. Укажите random seed = 7.**

In [4]:
%%time
!vw -i kaggle_data/users_model.vw -t -d kaggle_data/valid.vw \
-p kaggle_data/vw_valid_pred.csv --random_seed 7 --quiet 

CPU times: user 16.7 ms, sys: 0 ns, total: 16.7 ms
Wall time: 962 ms


**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [12]:
vw_valid_pred = pd.read_csv('kaggle_data/vw_valid_pred.csv', header=None)
res = accuracy_score(y_valid, vw_valid_pred)

**Какой получается доля правильных ответов на отложенной выборке? Запишите в файл *answer6_1.txt*, округлив до 3 знаков после запятой.**

In [11]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [14]:
write_answer_to_file("{}".format(round(res, 3)),
                     'answer6_1.txt')

**Обучите модель с теми же параметрами на всей обучающей выборке – *kaggle_data/train.vw*. При этом укажите новый кэш-файл (--cache_file).**

In [ ]:
%%time
!vw --oaa 550 ''' ВАШ КОД ЗДЕСЬ '''

**Сделайте прогноз для тестовой выборки.**

In [ ]:
%%time
!vw -t ''' ВАШ КОД ЗДЕСЬ '''

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

Прогноз, считанный из файла:

In [ ]:
vw_pred = ''' ВАШ КОД ЗДЕСЬ '''

Прогноз для отправки на Kaggle:

In [ ]:
vw_subm = class_encoder.inverse_transform ''' ВАШ КОД ЗДЕСЬ '''

In [ ]:
write_to_submission_file(vw_subm,
                         'kaggle_data/first_vw_submission.csv')

**Полученный результат соответствует бейзлайну "1st Vowpal Wabbit" на публичном лидерборде в нашем соревновании Kaggle.**

## Напутствие

Итак, мы познакомились с отличной библиотекой Vowpal Wabbit. Хотя это только начало: в VW реализованы матричные разложение и обучение с подкреплением, тематическое моделирование и активное обучение, name entity recognition и нейронные сети. Если Вам придется как-нибудь обучать модель на терабайте данных, при этом расходуя несколько мегабайт оперативной памяти, Вы наверняка вернетесь к этому тьюториалу (если, конечно, не выпустят библиотеку с более удобным интерфейсом). Хочется верить, что именно эта часть проекта оказалось самой полезной!

## Пути улучшения
На этой неделе опять дается время на соревнование, а также на оформление финального проекта.
Что еще можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели
 - Настроить параметры Vowpal Wabbit с hyperopt, попробовать другие функции потерь
 - Попробовать TF-IDF и n-граммы

На следующей, заключительной, неделе мы оформим всю работу над проектом в виде одного файла (.pdf или .ipynb) и будем проверять проекты друг друга.